In [1]:
import webotsgym as wg

from webotsgym.config import WebotConfig, SimSpeedMode
from webotsgym.environment import webotsgym, WebotsGrid
from webotsgym.evaluate import Evaluate, EvaluateMats, EvaluatePJ0
from webotsgym.action import DiscreteAction, ContinuousAction
from webotsgym.observation import Observation

import numpy as np

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
def exponential_decay(x, N0=1, lambda_=5):
    return N0*np.exp(-lambda_*x)

def exponential_penalty(x, step_penalty=-1, lambda_=5):
    return step_penalty * (1 - exponential_decay(x, lambda_=lambda_))

class MyEval(Evaluate):
    def __init__(self, env, config: WebotConfig = WebotConfig()):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        if self.env.gps_actual == self.env.gps_target:
            reward = 10000
        else:
            distance_normalized = self.env.get_target_distance(normalized=True)
            step_base_penalty = -1
            reward = exponential_penalty(x=distance_normalized, step_penalty=step_base_penalty, lambda_=3)
            if self.env.gps_visited_count > 3:
                reward -= 0.2 * (self.env.gps_visited_count - 2)**2
            if self.env.state._action_denied is True:
                reward -= 500
        return reward
        
        
    def check_done(self):
        if self.env.time_steps == 300:
            return True
        if self.env.total_reward < -10000:
            return True
        if self.env.get_target_distance(normalized=False) < 0.05:
            return True
        return False


config = WebotConfig()
config.sim_mode = SimSpeedMode.RUN
config.reset_env_after = 20000
config.num_obstacles = 12
config.world_size = 8
env = WebotsGrid(train=True, 
                 config=config,
                 evaluate_class = MyEval)

/home/fabian/.local/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Accepting on Port:  10201
sending: start env 1


In [ ]:
time_steps = 0
model_name = "PPO_webots_v1"

model = PPO1("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100)
model.save("models/keep/{}".format(model_name))

while time_steps < 500000:
    model = PPO1("MlpPolicy", env, verbose=1)
    model.load("models/keep/{}".format(model_name))
    model.learn(total_timesteps=100)
    model.save("models/keep/{}".format(model_name))
    time_steps += 100





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
********** Iteration 0 ************
sending: start env 1


In [ ]:
int(config.fast_simulation)